In [ ]:
import jupyter_black
jupyter_black.load(lab=False)

In [ ]:
with open("day7.txt", "r") as f:
    data = f.read()

In [ ]:
def get_folder(file_structure, path):
    path = path.strip("/")
    if path == "":
        return file_structure
    else:
        splitted = path.split("/")
        top = splitted.pop(0)
        path = "/".join(splitted)
        return get_folder(file_structure[top], path)


def update_current_dir(target_dir, current_dir):
    if target_dir == "..":
        return "/".join(current_dir.split("/")[:-2]) + "/"
    return current_dir + target_dir + "/"


def update_filestructure(filestructure, ls_result, current_dir):
    filestructure = filestructure.copy()
    if ls_result.startswith("dir"):
        _, directory = ls_result.split(" ")
        get_folder(filestructure, current_dir)[directory] = dict()
    else:
        size, name = ls_result.split(" ")
        get_folder(filestructure, current_dir)[name] = int(size)
    return filestructure


def map_out_file_system(commands):

    current_dir = "root/"
    keywords = ["cd", "ls"]
    file_structure = {"root": dict()}

    for command in commands:
        command = command.strip()
        if command.startswith("cd"):
            _, target_dir = command.split(" ")
            current_dir = update_current_dir(target_dir, current_dir)
        elif command.startswith("ls"):
            for ls_result in command.split("\n")[1:]:
                file_structure = update_filestructure(
                    file_structure, ls_result, current_dir
                )

    return file_structure


def get_filesizes(file_structure):
    file_sizes = dict()

    def add_sizes(folder, top_folders):
        for key, val in folder.items():
            if type(val) is dict:
                if len(top_folders) > 0:
                    key = top_folders[-1] + "/" + key
                add_sizes(val, top_folders + [key])
            elif type(val) == int:
                for fol in top_folders:
                    file_sizes[fol] = file_sizes.get(fol, 0) + val

    add_sizes(file_structure, [])
    return file_sizes

In [ ]:
fs = map_out_file_system(data.split("$")[2:])

file_sizes = get_filesizes(fs)
sum([size for _, size in file_sizes.items() if size <= 100000])

In [ ]:
free_space = 70000000 - file_sizes["root"]
size_needed = 30000000 - free_space
min([size for _, size in file_sizes.items() if size > size_needed])